In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from joblib import Parallel, delayed
from sklearn.svm import SVC

In [2]:
data = pd.read_csv('projet 2.csv')

C:\Users\chabc\AppData\Local\Temp\ipykernel_8188\1692616215.py:1: DtypeWarning: Columns (5,11) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('projet 2.csv')


In [3]:
nouvelle_data = data.loc[:, ['Problème', 'Société' ,"RéP-l'entreprise" ]]

In [4]:
societe_id_mapping = {societe: idx + 1 for idx, societe in enumerate(nouvelle_data['Société'].unique())}

In [5]:
probleme_id_mapping = {probleme: idy + 1 for idy, probleme in enumerate(nouvelle_data['Problème'].unique())}

In [6]:
# Ajouter une colonne 'id_produit' en utilisant le mapping
nouvelle_data['id_societe'] = nouvelle_data['Société'].map(societe_id_mapping)

In [7]:
# Ajouter une colonne 'id_produit' en utilisant le mapping
nouvelle_data['id_probleme'] = nouvelle_data['Problème'].map(probleme_id_mapping)

In [8]:
nouvelle_data_non_null = nouvelle_data.dropna(subset=["RéP-l'entreprise"])

In [9]:
nouvelle_data.shape

(670598, 5)

In [10]:
nouvelle_data_non_null.head(749)

,Problème,Société,RéP-l'entreprise,id_societe,id_probleme
12005,False statements or representation,"FAIR COLLECTIONS & OUTSOURCING, INC.",Company chooses not to provide a public response,306,40
12324,Improper contact or sharing of info,Professional Recovery Management,Company believes it acted appropriately as aut...,756,28
12372,Communication tactics,Focus Holding Company,Company believes it acted appropriately as aut...,420,9
17494,"Loan servicing, payments, escrow account",Bank of America,Company has responded to the consumer and the ...,4,5
18105,Incorrect information on credit report,Experian,Company chooses not to provide a public response,12,13
...,...,...,...,...,...
60668,Managing the loan or lease,Headwaters Financial Corp.,Company believes it acted appropriately as aut...,1804,1
60669,Closing/Cancelling account,Citibank,Company chooses not to provide a public response,7,23
60670,Unable to get credit report/credit score,Experian,Company chooses not to provide a public response,12,22
60687,Disclosure verification of debt,"BWW Law Group, LLC",Company chooses not to provide a public response,1805,29


In [11]:
X = nouvelle_data_non_null[['id_societe', 'id_probleme']]
y = nouvelle_data_non_null["RéP-l'entreprise"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
# Encoder les variables catégorielles sur l'ensemble d'entraînement seulement
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)

In [13]:
# Créer un modèle de forêt aléatoire
model = RandomForestClassifier()

In [14]:
model.fit(X_train, y_train_encoded)

RandomForestClassifier()

In [21]:
# Filtrer X_test et y_test pour ne conserver que les valeurs présentes dans l'ensemble d'entraînement
#cette partie du code assure que seules les données de test compatibles avec l'ensemble d'entraînement sont utilisées pour évaluer les performances du modèle, ce qui garantit une évaluation juste et cohérente.
# ichouf eli jeweb alihom bel shih wale 
filtered_X_test = X_test[y_test.isin(y_train)]
filtered_y_test = y_test[y_test.isin(y_train)]

In [16]:
# Évaluer les performances du modèle sur les données filtrées
rf_accuracy = model.score(filtered_X_test, label_encoder.transform(filtered_y_test))
print("Accuracy with Random Forest on filtered test set:", rf_accuracy)

Accuracy with Random Forest on filtered test set: 0.5886363636363636


In [17]:
# Prédire les valeurs manquantes
missing_data = nouvelle_data[nouvelle_data["RéP-l'entreprise"].isna()]
X_missing = missing_data[['id_societe', 'id_probleme']]
predicted_values_encoded = model.predict(X_missing)
predicted_values = label_encoder.inverse_transform(predicted_values_encoded)

In [18]:
# Remplacer les valeurs manquantes dans le DataFrame d'origine
nouvelle_data.loc[nouvelle_data["RéP-l'entreprise"].isna(), "RéP-l'entreprise"] = predicted_values

In [19]:
nouvelle_data.to_csv("dataset_rempli_specific_rep_entreprise.csv", index=False)

In [20]:
nouvelle_data.to_excel("dataset_rempli_specific_Frep_entreprise.xlsx", index=False)